<a href="https://colab.research.google.com/github/nathencinoza/pyspark-movie-dataset/blob/main/S14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

S14) ¿Cuál es el usuario cuya velocidad promedio para hacer ratings es mayor?


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 63 kB/s 
     |████████████████████████████████| 199 kB 56.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b2a4546b1dc459305f43484ac603428afbb0997906f96e3ddadaf43b0ca14305
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  I

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
rdd = sqlContext.read.parquet('/content/drive/MyDrive/Orga 2/Movie dataset/Parquets/ratings.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
rdd.first()

Row(userId=1, movieId=110, rating=1.0, timestamp=1425941529)

In [ ]:
rdd1 = rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a+b)

In [ ]:
rdd2 = rdd.map(lambda x: (x[0], (x[3], x[3]))).reduceByKey(lambda x,y: (x[0] if(x[0] < y[0]) else y[0], x[1] if(x[1] > y[1]) else y[1])).map(lambda x: (x[0], x[1][1]-x[1][0]))

Las tuplas quedan de la forma: (userId, (cantidad de reviews, max timestamp - min timestamp))

In [ ]:
rdd3 = rdd1.join(rdd2)
rdd3.take(5)

[(4, (62, 7041)),
 (8, (113, 1792)),
 (12, (248, 432871)),
 (16, (182, 27516)),
 (20, (270, 124694296))]

Usuario cuya velocidad promedio para hacer ratings es mayor:

In [ ]:
rdd3.filter(lambda x: x[1][1] != 0).map(lambda x: (x[0], x[1][0]/x[1][1])).max(key=lambda x: x[1])

(268565, 269.0)